## User Experience Analytics

### Import Liberaries

In [9]:
import pickle
import numpy as np
import pandas as pd
from math import floor
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import zscore
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, normalize
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import matplotlib_inline 
import sys, os

In [10]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option("expand_frame_repr", False)
pd.set_option('display.float_format', '{:.2f}'.format)

In [11]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from data_visualizer import *
from data_selecter import *
from outlier_handler import OutlierHandler

### Loading Data

In [12]:
df1 = pd.read_csv("../src/data/my_clean_data2.csv")
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146887 entries, 0 to 146886
Data columns (total 55 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   Unnamed: 0                        146887 non-null  int64  
 1   Bearer Id                         146887 non-null  int64  
 2   Start                             146887 non-null  object 
 3   Start ms                          146887 non-null  float64
 4   End                               146887 non-null  object 
 5   End ms                            146887 non-null  float64
 6   IMSI                              146887 non-null  int64  
 7   MSISDN/Number                     146887 non-null  int64  
 8   IMEI                              146887 non-null  int64  
 9   Last Location Name                146887 non-null  object 
 10  Avg RTT DL (ms)                   146887 non-null  float64
 11  Avg RTT UL (ms)                   146887 non-null  f

1. Aggregate, per customer, the following information (treat missing & outliers by replacing by the mean or the mode of the corresponding variable):
-	Average TCP retransmission
-	Average RTT
-	Handset type
-	Average throughput


In [13]:
user_experience = df1[[
    "MSISDN/Number",
    "Avg RTT DL (ms)",
    "Avg RTT UL (ms)",
    "Avg Bearer TP DL (kbps)",
    "Avg Bearer TP UL (kbps)",
    "TCP DL Retrans. Vol (Bytes)",
    "TCP UL Retrans. Vol (Bytes)",
    "Handset Type"]].copy()

In [14]:
user_experience['total_avg_rtt'] = user_experience['Avg RTT DL (ms)'] + user_experience['Avg RTT UL (ms)']
user_experience['total_avg_tp'] = user_experience['Avg Bearer TP DL (kbps)'] + user_experience['Avg Bearer TP UL (kbps)']
user_experience['total_avg_tcp'] = user_experience['TCP DL Retrans. Vol (Bytes)'] + user_experience['TCP UL Retrans. Vol (Bytes)']
user_experience.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146887 entries, 0 to 146886
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   MSISDN/Number                146887 non-null  int64  
 1   Avg RTT DL (ms)              146887 non-null  float64
 2   Avg RTT UL (ms)              146887 non-null  float64
 3   Avg Bearer TP DL (kbps)      146887 non-null  float64
 4   Avg Bearer TP UL (kbps)      146887 non-null  float64
 5   TCP DL Retrans. Vol (Bytes)  146887 non-null  float64
 6   TCP UL Retrans. Vol (Bytes)  146887 non-null  float64
 7   Handset Type                 146887 non-null  object 
 8   total_avg_rtt                146887 non-null  float64
 9   total_avg_tp                 146887 non-null  float64
 10  total_avg_tcp                146887 non-null  float64
dtypes: float64(9), int64(1), object(1)
memory usage: 12.3+ MB


In [15]:
user_experience.head()

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type,total_avg_rtt,total_avg_tp,total_avg_tcp
0,33664962560,42.00,5.00,23.00,44.00,19520.00,7230.00,Samsung Galaxy A5 Sm-A520F,47.00,67.00,26750.00
1,33681854464,65.00,5.00,16.00,26.00,19520.00,7230.00,Samsung Galaxy J5 (Sm-J530),70.00,42.00,26750.00
2,33760626688,65.00,5.00,6.00,9.00,19520.00,7230.00,Samsung Galaxy A8 (2018),70.00,15.00,26750.00
3,33750343680,65.00,5.00,44.00,44.00,19520.00,7230.00,undefined,70.00,88.00,26750.00
4,33699794944,65.00,5.00,6.00,9.00,19520.00,7230.00,Samsung Sm-G390F,70.00,15.00,26750.00


In [16]:
_user_experience = user_experience.groupby('MSISDN/Number').agg({
    'total_avg_rtt': 'sum',
    'total_avg_tp': 'sum',
    'total_avg_tcp': 'sum',
    'Handset Type': [lambda x: x.mode()[0]]})

user_experience = pd.DataFrame(columns=[
    "total_avg_rtt",
    "total_avg_tp",
    "total_avg_tcp",
    "Handset Type"])

user_experience["total_avg_rtt"] = _user_experience["total_avg_rtt"]['sum']
user_experience["total_avg_tp"] = _user_experience["total_avg_tp"]['sum']
user_experience["total_avg_tcp"] = _user_experience["total_avg_tcp"]['sum']
user_experience["Handset Type"] = _user_experience["Handset Type"]['<lambda>']
user_experience.head()

,total_avg_rtt,total_avg_tp,total_avg_tcp,Handset Type
MSISDN/Number,,,,
33601001472,77.00,175.00,10581256.50,Apple iPhone 7 (A1778)
33601003520,59.00,97.00,4150403.00,undefined
33601007616,203.00,43452.50,8049700.50,Apple iPhone Se (A1723)
33601009664,101.00,7908.00,43952.00,Samsung Galaxy A8 (2018)
33601011712,105.00,43760.00,130637.00,Huawei Mate 10 Pro Porsche Design Huawei Mate 10


2. Compute & list 10 of the top, bottom and most frequent:
- TCP values in the dataset. 
- RTT values in the dataset.
- Throughput values in the dataset


In [17]:
#TCP values in the dataset.
sorted_by_tcp = user_experience.sort_values(
    'total_avg_tcp', ascending=False)
top_10 = sorted_by_tcp.head(10)['total_avg_tcp']
last_10 = sorted_by_tcp.tail(10)['total_avg_tcp']
most_10 = user_experience['total_avg_tcp'].value_counts().head(10)


In [18]:
from IPython.display import Image
import plotly.io as pio

In [19]:
#RTT values in the dataset. 
sorted_by_rtt = user_experience.sort_values(
    'total_avg_rtt', ascending=False)
top_10 = sorted_by_rtt.head(10)['total_avg_rtt']
last_10 = sorted_by_rtt.tail(10)['total_avg_rtt']
most_10 = user_experience['total_avg_rtt'].value_counts().head(10)

In [20]:
#Throughput values in the dataset
sorted_by_tp = user_experience.sort_values(
    'total_avg_tp', ascending=False)
top_10 = sorted_by_tp.head(10)['total_avg_tp']
last_10 = sorted_by_tp.tail(10)['total_avg_tp']
most_10 = user_experience['total_avg_tp'].value_counts().head(10)

3. Compute & report:
- The distribution of the average throughput  per handset type and provide interpretation for your findings.
- The average TCP retransmission view per handset type and provide interpretation for your findings.
 

In [21]:
handset_type_df = user_experience.groupby('Handset Type').agg(
    {'total_avg_tp': 'mean', 'total_avg_tcp': 'mean'})
handset_type_df.head()

,total_avg_tp,total_avg_tcp
Handset Type,,
A-Link Telecom I. Cubot A5,23510.00,7673105.50
A-Link Telecom I. Cubot Note Plus,43770.00,25060970.00
A-Link Telecom I. Cubot Nova,59206.50,191163.00
A-Link Telecom I. Cubot X18 Plus,4351.00,293.00
Acer Liquid Z630,4902.00,803505.00


In [22]:
sorted_by_tp = handset_type_df.sort_values(
    'total_avg_tp', ascending=False)
top_tp = sorted_by_tp['total_avg_tp']

- We can see modern phones have higher throughput.

In [23]:
#The average TCP retransmission view per handset type and provide interpretation for your findings.
sorted_by_tcp = handset_type_df.sort_values(
    'total_avg_tcp', ascending=False)
top_tcp = sorted_by_tcp['total_avg_tcp']

4. Using the experience metrics above, perform a k-means clustering (where k = 3) to segment users into groups of experiences and provide a brief description of each cluster. (The description must define each group based on your understanding of the data)

In [24]:
experiance_metric_df = user_experience[[
    "total_avg_rtt",
    "total_avg_tp",
    "total_avg_tcp"]].copy()
experiance_metric_df.head()

,total_avg_rtt,total_avg_tp,total_avg_tcp
MSISDN/Number,,,
33601001472,77.00,175.00,10581256.50
33601003520,59.00,97.00,4150403.00
33601007616,203.00,43452.50,8049700.50
33601009664,101.00,7908.00,43952.00
33601011712,105.00,43760.00,130637.00


In [25]:
df_outliers = OutlierHandler(experiance_metric_df)
df_outliers.getOverview(['total_avg_rtt','total_avg_tp','total_avg_tcp'])

Name of columns,total_avg_rtt,total_avg_tp,total_avg_tcp
Min,0.00,0.00,97.00
Q1,65.00,194.00,368828.00
Median,134.00,15090.00,3594796.50
Q3,305.00,53799.25,10174021.38
Max,5852.00,1910626.50,175947323.00
IQR,240.00,53605.25,9805193.38
Lower fence,-295.00,-80213.88,-14338962.06
Upper fence,665.00,134207.12,24881811.44
Skew,2.61,3.41,2.86
Number_of_outliers,4008,3963,3404


In [26]:
df_outliers.replace_outliers_with_fences(
    ["total_avg_rtt",
     "total_avg_tp",
     "total_avg_tcp"])
#df_outliers = DfOutlier(experiance_metric_df)
df_outliers.getOverview(["total_avg_rtt",
     "total_avg_tp",
     "total_avg_tcp"])

Name of columns,total_avg_rtt,total_avg_tp,total_avg_tcp
Min,0.00,0.00,97.00
Q1,65.00,194.00,368828.00
Median,134.00,15090.00,3594796.50
Q3,305.00,53799.25,10174021.38
Max,665.00,134207.12,24881811.44
IQR,240.00,53605.25,9805193.38
Lower fence,-295.00,-80213.88,-14338962.06
Upper fence,665.00,134207.12,24881811.44
Skew,1.15,1.09,1.11
Number_of_outliers,0,0,0


In [27]:
scaler = StandardScaler()
scaled_array = scaler.fit_transform(df_outliers.df)
scaled_array
pd.DataFrame(scaled_array).head(5)

,0,1,2
0,-0.69,-0.80,0.43
1,-0.78,-0.80,-0.36
2,-0.09,0.13,0.12
3,-0.58,-0.64,-0.86
4,-0.56,0.14,-0.85


In [28]:
data_normalized = normalize(scaled_array)
pd.DataFrame(data_normalized).head(5)

,0,1,2
0,-0.61,-0.70,0.37
1,-0.66,-0.68,-0.31
2,-0.47,0.66,0.59
3,-0.48,-0.52,-0.71
4,-0.54,0.13,-0.83


In [29]:
kmeans = KMeans(n_clusters=3, random_state=0).fit(data_normalized)
kmeans.labels_

array([1, 1, 2, ..., 0, 1, 1], shape=(34910,), dtype=int32)

In [30]:
experiance_metric_df.insert(0, 'cluster', kmeans.labels_)
experiance_metric_df

,cluster,total_avg_rtt,total_avg_tp,total_avg_tcp
MSISDN/Number,,,,
33601001472,1,77.00,175.00,10581256.50
33601003520,1,59.00,97.00,4150403.00
33601007616,2,203.00,43452.50,8049700.50
33601009664,1,101.00,7908.00,43952.00
33601011712,0,105.00,43760.00,130637.00
...,...,...,...,...
33789960192,1,41.00,106.00,221818.00
33789966336,1,53.00,67.00,56578.00
33789980672,0,109.00,52859.00,7835020.50


In [31]:
experiance_metric_df['cluster'].value_counts()

cluster
1    19573
2    10411
0     4926
Name: count, dtype: int64

In [51]:
fig = px.scatter(experiance_metric_df, x='total_avg_rtt', y="total_avg_tp",
                 color='cluster', size='total_avg_tcp')


In [33]:
# Save the final data frame
user_experience.to_csv('../src/data/TellCo_user_experience_data.csv')

In [34]:
with open("../models/TellCo_user_experiance.pkl", "wb") as f:
    pickle.dump(kmeans, f)